# Unfolding of an Experimental 16 MeV on Be PHS

This notebook contains the specific commands used to take the 16 MeV on Be experimental data from Root TTrees and turn them into the correctly formatted .phs and .rsp files for the the NSD and HEPROW unfolding programs.  It will also generate the input neutron spectrum to use in comparison.  

This notebook is broken into the following sections: <br\>
1) Calibration of PHS <br\>
2) Response Matrix Generation <br\>
3) HEPROW Unfold <br\>
4) NSD Unfold <br\>
5) Results <br\>

Root must be installed and PyRoot enabled for this notebook to work.

First, load the necessary libraries, set the appropriate environment variables, and compile the C++ support macros from the instructions @ https://bitbucket.org/berkeleylab/nsd-rootscripts/wiki/LibraryCompilation.

Load python dependencies:

In [1]:
from ROOT import gROOT
import numpy as np

gROOT.ProcessLine('HistogramWriter writer;')
gROOT.ProcessLine('HistogramOperations ops;')

47062041833520L

## PHS Calibration

### User inputs

dataPath = the absolute location of where the root files are located <br/>
phsName = file name for the pulse height spectra TTree <br/>
outPath = the absolute path of where to place the generated HEPROW input files

In [2]:
dataPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/"
phsName="Experiments/PHS/16MeVBe/SinglePh.root"
outPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/16MeVBe/Unfold/HEPROW/Inputs/"

### Load PHS Data File

In [5]:
gROOT.ProcessLine('TFile f("{}")'.format(dataPath+phsName))

47062041833568L

### Apply Calibration and print the .phs file

The calibration constants used were:  

a = 0.00327102 <br/>
b = -0.427562 <br/>
Ec = 0.0531585  <br/>
E1 = 0.0310429 <br/>
E2 = 0.0151981 <br/>

In [6]:
gROOT.ProcessLine('ops.applyCalibration(histo3, 0.00327102, -0.427562);')

0L

In [7]:
gROOT.ProcessLine('histo3->Draw()')

0L

Truncate all negative and all zeros data.  This is necessary for unfolding in HEPROW.

Find the first bin with data.  The cell will print the first bin, but not the lower MeVee edge.  It is a bit clunky, but you can read it directly from the terminal running this notebook (or run the commands in Root directly).

In [8]:
gROOT.ProcessLine('tmpSpectrum = (TH1D*)ops.truncateHist(histo3,0.105,14);')

i=0
while gROOT.ProcessLine('tmpSpectrum->GetBinContent({})'.format(i))==0:
    i+=1
print i

gROOT.ProcessLine('tmpSpectrum->GetXaxis()->GetBinLowEdge({})'.format(i))

64


0L

Choose a rebin number to get the desired MeVee/bin.  In this case, 16 was chosen to get ~ 0.05 MeVee/bin 

In [9]:
rebin=16
gROOT.ProcessLine('tmpSpectrum->Rebin({})'.format(rebin))
gROOT.ProcessLine('PulseHeightSpectrum = (TH1D*)ops.truncateHist(tmpSpectrum,3.084175e-01,14);')
gROOT.ProcessLine('PulseHeightSpectrum->GetXaxis()->GetBinLowEdge({})'.format(0))
gROOT.ProcessLine('PulseHeightSpectrum->Draw()')

TH1D::Rebin:0: RuntimeWarning: ngroup=16 is not an exact divider of nbins=3935.


0L

Print out the .phs file for both statistic bins and the intrinsic rebin from above:

In [11]:
gROOT.ProcessLine('TH1* dataHist = ops.rebinStatistically(PulseHeightSpectrum,100);')
gROOT.ProcessLine('writer.PhToHEPROW(PulseHeightSpectrum,"{}histo3_phs_05")'.format(outPath))
gROOT.ProcessLine('writer.PhToHEPROW(dataHist,"{}histo3_stat_100_phs_05")'.format(outPath))

0L

### Ouput the Initial Spectrum

Uses the simulated 16 MeV spectrum, which was created from Meulders

In [ ]:
gROOT.ProcessLine('TFile g("/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Simulated/PHS/16MeVBe/meulders16BeSimulation.root)')
gROOT.ProcessLine('primary->Draw("nEn>>initSpec(20,0,20)")')
gROOT.ProcessLine('writer.Th1ToAscii(initSpec,"{}inputSpectrum_1")'.format(outPath))

## Response Matrix

### Response Matrix Inputs

rspName = file name for the response matrix TTree <br/>

rspEwidth = the width of bins to use for the binning of the response matrix energy <br/>
rspEmin = the minimum Energy to use for the binning of the response matrix  <br/>
rspEmax = the maximum Energy to use for the binning of the response matrix  <br/>
rspLwidth = the width of bins to use for the binning of the response matrix light <br/>
rspLmin = the minimum Light to use for the binning of the response matrix  <br/>
rspLmax = the maximum Light to use for the binning of the response matrix  <br/>

In [4]:
rspName="Simulated/PHS/ResponseMatrices/simSideResponse20Mil.root"

rspEwidth=1
rspEmin=0.0
rspEmax=25.0
rspLwidth=0.05
rspLmin=0.0
rspLmax=16.0

### Load Response Matrix Data File

In [12]:
gROOT.ProcessLine('SimulationManipulation sm("{}",0)'.format(dataPath+rspName))

47062041834384L

### Create the bin structures and print .rsp file

In [13]:
# Create the bin structures
rspEbins=np.arange(rspEmin,rspEmax,rspEwidth)
rspEbins=np.append(rspEbins,rspEmax)
#print rspEbins
rspLbins=np.arange(rspLmin,rspLmax,rspLwidth)
rspLbins=np.append(rspLbins,rspLmax)
#print rspLbins
gROOT.ProcessLine('const Int_t EBINS = {}; const Int_t LBINS = {};'.format(len(rspEbins)-1,len(rspLbins)-1))
gROOT.ProcessLine('Double_t eEdges[EBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspEbins),"}"))
gROOT.ProcessLine('Double_t lEdges[LBINS + 1] = {}{}{};'.format("{",", ".join(str(e) for e in rspLbins),"}"))
gROOT.ProcessLine('axis1 = TAxis(EBINS,eEdges);')
gROOT.ProcessLine('axis2 = TAxis(LBINS,lEdges);')

# Create the Histogram and output file
gROOT.ProcessLine('TH2* hist=sm.getNormalizedResponseMatrix(axis1,axis2)')
gROOT.ProcessLine('hist->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(hist,"{}resp_05_1")'.format(outPath))

0L

### Smear the Response Matrix and Create the .rsp File

In [14]:
gROOT.ProcessLine('TH2* smearHist2 = ops.skewedGausSmearMatrix(hist,0.0531585, 0.0310429, 0.0151981)')
gROOT.ProcessLine('smearHist2->Draw("colz")')
gROOT.ProcessLine('writer.ResponseToHEPROW(smearHist2,"{}smearedResp_05_1")'.format(outPath))

0L

## HEPROW Unfold

The actually running of HEPROW is done separately through the Windows executables.  The location for the files is: <br\>

/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/16MeVBe/Unfold/HEPROW

### Mik.inp Support

These lines are useful to develop the mik.inp file:

In [15]:
bounds = np.linspace(1.0,rspEmax-1,len(rspEbins)-3)   #(First right bin boundary, last right bin, boundary, number of bins)
print len(bounds)
for i in range(0,len(bounds)):
    print '{}                                      right boundary of energy interval {}'.format(bounds[i],i+1)


19
1.0                                     right boundary of 1 energy interval
2.0                                     right boundary of 2 energy interval
3.0                                     right boundary of 3 energy interval
4.0                                     right boundary of 4 energy interval
5.0                                     right boundary of 5 energy interval
6.0                                     right boundary of 6 energy interval
7.0                                     right boundary of 7 energy interval
8.0                                     right boundary of 8 energy interval
9.0                                     right boundary of 9 energy interval
10.0                                     right boundary of 10 energy interval
11.0                                     right boundary of 11 energy interval
12.0                                     right boundary of 12 energy interval
13.0                                     right boundary of 13 energy interval
1

## NSD Unfold

This uses the unfolding algorithms developed by Matt Harasty and Josh Brown contained in the nsd-rootscripts library to unfold the same data.  

First we must load the script to do the unfolding. NOTE: All of the parameters set in this notebook above are done explicitly within this script.  It is recommended that a unique script be made in each unfold directory to document the settings used.

NOTE: Do not run the ROOT import cell if it was run about and still active in the kernel.  It is repeated here for convience when not doing the HEPROW processing.

In [ ]:
from ROOT import gROOT

gROOT.ProcessLine('HistogramWriter writer;')

In [ ]:
gROOT.ProcessLine('.L /home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/16MeVBeUnfold.cpp')

### Run the Algorithm

Create the unfolding object, run the unfolding algorithm, and plot the results:

In [ ]:
gROOT.ProcessLine('NeutronSpectrumUnfolding* obj = setup1MeVUnfolding()')
gROOT.ProcessLine('obj->runRootMinimization("Minuit")')
gROOT.ProcessLine('obj->drawState()')

### Write the results 

Save to both a root file and .phs file:

In [ ]:
gROOT.ProcessLine('obj->writeState("/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/NSDUnfolded16MeV_Cal3_1.root")')
gROOT.ProcessLine('TH1D* outputHisto = obj->getCurrentTrialClone()')
gROOT.ProcessLine('writer.Th1ToAscii(outputHisto, "/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Unfolding/Experiments/16MeVBe/Unfold/NSD16MeVSpectrum_Cal3_1")')

In [ ]:
#Unfolding with error process
In the compile directory for nsd-rootscripts:
1) Modify RunMCUnfoldError.cpp to have the correct variables.   
2) make RunMCUnfoldError.exe
3) ./RunMCUnfoldError.exe

## Results

This section contains all of the output post-processing to end up with a plot of the spectrum.

In [1]:
import sys
import os

import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline

# Path to support scripts 
sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/DataAnalysis'))
from DataManipulation import bin_differentiation, normAUBC
from DataIO import read_delimited_data_file
from Histograms import Histogram

sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/Unfolding'))
from HEPROW import readMTX, readGru, readFlu
from Root import FluxNormalization

sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/GeneralNuclear'))
from Detectors import nonparalyzable_beam_dead_time, nonparalyzable_dead_time

### Set normalization parameters

Based on the detector setup and readings from the current integrator and current monitor reported earlier in this notbook.

In [3]:
norm = FluxNormalization()
norm.runTime=48*60.+48
norm.currentMonitor = 0.99E-3*norm.runTime
norm.currentIntegrator = 0.66E-3*norm.runTime
norm.set_solid_angle(640+36.3, 25)
norm.set_dead_time(nonparalyzable_beam_dead_time, obsCountRate=(8576894)/norm.runTime, 
                   tauDetector=1.25E-6, tauBeam=158E-9)
print str(norm)


Normalization Parameters
Total Run Time = 2928.0 s
Current Monitor Inegrated Current = 2.89872 microC
Current Integrator Reading = 1.93248 microC
Solid Angle = 5.46589436057e-05 sr
Fractional Dead Time = 0.00340220869343



### Set Data Locations

In [ ]:
# HEPROW Inputs:
heprowPath = "/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/16MeVBe/Unfold/HEPROW/"

heprowName = "mik_1.gru"
unfanaName = "unf_1.gru"
faltwPHSName = "faltw_1.phs"
measPHSName = "Inputs/histo3_stat_100_phs_05.phs"
mtxName = "MIEKE_1.MTX"
heprowBinBounds = "low"

# Meulders - 16 MeV d on Be 
meuldersPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/16MeVBe/Unfold/"

meuldersName="Meulders_1.txt"
meuldersBinBounds="mid"

# Lone
nsdPath="/home/pyne-user/Dropbox/UCB/Research/ETAs/88Inch/Data/Experiments/PHS/16MeVTa/Unfold/"

nsdName="Lone_1.txt"
nsdBinBounds="low"